In [2]:
import spacy

nlp = spacy.load("en_core_web_lg")

In [3]:
import pandas as pd

# query = "Query_NYT_5y_surveillance"
query = "Query_NYT_13y_surveillance"
# query = "Query_US_5y_Svalbard"

CSV_FILE = query + ".csv"
FEATHER_FILE = query + ".feather"

df = pd.read_feather(FEATHER_FILE)

df.sample(3)

,stories_id,title,url,year,text,text_downloaded,tags
2734,1036338766,"Migrant Caravan, Transgender Rights, Khashoggi...",https://www.nytimes.com/2018/10/22/briefing/ca...,2018,"Supported by\nMigrant Caravan, Transgender Rig...",True,"[{'label': 'immigration and refugees', 'score'..."
50,1140964775,Government Shutdown Curtails F.D.A. Food Inspe...,https://www.nytimes.com/2019/01/09/health/shut...,2019,Supported by\nGovernment Shutdown Curtails F.D...,True,"[{'label': 'food contamination and poisoning',..."
1665,2572965622,"In Biden’s Unannounced Visit to Kyiv, a Previe...",https://www.nytimes.com/2023/02/20/us/politics...,2023,Supported by\nIn Biden’s Unannounced Visit to ...,True,"[{'label': 'politics and government', 'score':..."


In [4]:
from tqdm.notebook import tqdm

GPE = [""] * len(df)
IND = [""] * len(df)
ORG = [""] * len(df)

# Browse the data ...
for index, row in tqdm(df.iterrows(), total=len(df)):
    text = row.text
    doc = nlp(text)

    entities = {
        "GPE": [],
        "IND": [],
        "ORG": [],
    }

    for ent in (e for e in doc.ents if e.label_ == "GPE"):
        entities["GPE"].append(ent.text)

    for ent in (e for e in doc.ents if e.label_ == "PERSON"):
        entities["IND"].append(ent.text)

    for ent in (e for e in doc.ents if e.label_ == "ORG"):
        entities["ORG"].append(ent.text)

    GPE[index] = entities["GPE"]
    IND[index] = entities["IND"]
    ORG[index] = entities["ORG"]

  0%|          | 0/3357 [00:00<?, ?it/s]

In [5]:
df['GPE'] = GPE
df['IND'] = IND
df['ORG'] = ORG

df.to_feather(FEATHER_FILE)